# Assignment 3 - Transliteration

### Setup [do not change]

In [1]:
!git clone https://github.com/butoialexandra/eth-nlp-f22-hw3.git

import os
os.chdir('eth-nlp-f22-hw3')

!pip install -e .

fatal: destination path 'eth-nlp-f22-hw3' already exists and is not an empty directory.
Obtaining file:///Users/simon/repos/nlp-assignment/ass3/eth-nlp-f22-hw3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: rayuela
    Found existing installation: rayuela 0.1
    Uninstalling rayuela-0.1:
      Successfully uninstalled rayuela-0.1
  Running setup.py develop for rayuela


In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

--2023-01-13 14:01:20--  https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC
Resolving docs.google.com (docs.google.com)... 2a00:1450:400a:802::200e, 172.217.168.46
Connecting to docs.google.com (docs.google.com)|2a00:1450:400a:802::200e|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/asa16ove1fela9t8mdj09k3ug3c4vcqc/1673614875000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=ee9db9ee-d7dd-4fde-b46c-ba832d8c68cf [following]
--2023-01-13 14:01:21--  https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/asa16ove1fela9t8mdj09k3ug3c4vcqc/1673614875000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=ee9db9ee-d7dd-4fde-b46c-ba832d8c68cf
Resolving doc-0s-50-docs.googleusercontent.com (doc-0s-50-docs.googleusercontent.com)... 2a00:1450:400a:802::2001, 1

In [3]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.base.misc import epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict

### Subquestion a) - Log Semiring

In [4]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        # TODO: implement Kleene star
        if self.score <= 0:
            return np.log(1 - np.exp(self.score))
        else:
            return np.inf

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        # TODO: implement log addition
        # For numerical stability you should use the using the log-sum-exp trick
        # https://en.wikipedia.org/wiki/LogSumExp
        return max(self.score, other.score)

    def __mul__(self, other):
        # TODO: implement log multiplication
        return self.score + other.score

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Log.zero = Log(-np.inf)
Log.one = Log(0)

# i = Log(Log.one)
# j = Log(1.0)

# print(i + j)

Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [5]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [6]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [7]:
def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    one, zero = semiring.one, semiring.zero # semiring one and zero
    fst = FST(semiring)
    
    for i, char in enumerate(s):
        fst.add_arc(State(i), Sym(char), Sym(char), State(i+1), one)

    # initial and final weights
    fst.set_I(State(0), one)
    fst.set_F(State(len(s)), one)
    
    return fst

In [8]:
# you can visualise the transducer
fst = string_fst("abc", semiring=Real)
fst

WFSA(4 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion c) - Edit Distance Transducer

As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer). 

In [9]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target 
    alphabets) and a semiring and returns an edit-distance transducer
    """
    
    one, zero = semiring.one, semiring.zero # semiring one and zero
    fst = FST(semiring)
    weight_delete = one
    weight_insert = one
    weight_sub = one
    eps = ε

    base = "BOS"
    fst.set_F(State(base), zero)
    
    # deletion
    for s in source_alphabet:
        fst.add_arc(State(base), Sym(s), Sym(eps), State(base), weight_delete)
    # insertion
    for t in target_alphabet:
        fst.add_arc(State(base), Sym(eps), Sym(t), State(t), weight_insert)
        
    for s in source_alphabet:
        for t in target_alphabet:
            fst.add_arc(State(base), Sym(s), Sym(t), State(t), weight_sub)

    # TODO: add arcs, initial and final weights
    for base in target_alphabet:
        
        fst.set_F(State(base), zero)
        
        # deletion
        for s in source_alphabet:
            fst.add_arc(State(base), Sym(s), Sym(eps), State(base), weight_delete)
        # insertion
        for t in target_alphabet:
            fst.add_arc(State(base), Sym(eps), Sym(t), State(t), weight_insert)
            
        for s in source_alphabet:
            for t in target_alphabet:
                fst.add_arc(State(base), Sym(s), Sym(t), State(t), weight_sub)
    
    # initial and final weights
    fst.set_I(State("BOS"), one)

    return fst

In [10]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Real)
fst

WFSA(3 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [11]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [12]:
# FST 1
examples[0]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [13]:
# FST 2
examples[0]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [14]:
# composition
examples[0]['composition']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [15]:
# semiring
examples[0]['semiring']

rayuela.base.semiring.Real

In [16]:
def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)
    eps = ε
    
    # TODO: implement composition

    for q1 in fst1.Q:
        for q2 in fst2.Q:
            for (i1, o1), q1p, w1 in fst1.arcs(q1):
                if o1 == ε:
                    comp_fst.add_arc(PairState(q1, q2), i1, o1, PairState(q1p, q2), w1)
                for (i2, o2), q2p, w2 in fst2.arcs(q2):
                    if i2 == ε:
                        comp_fst.add_arc(PairState(q1, q2), i2, o2, PairState(q1, q2p), w2)
                    if o1 == i2:
                        comp_fst.add_arc(PairState(q1, q2), i1, o2, PairState(q1p, q2p), w1 * w2)
    # print(comp_fst.Q)
    for s in comp_fst.Q:
        q1, q2 = s.state1, s.state2
        # print(q1, q2)
        if fst1.λ[q1] != fst1.R.zero:
            comp_fst.set_I(PairState(q1, q2), fst1.λ[q1] * fst2.λ[q2])
        if fst1.ρ[q1] != fst1.R.zero:
            comp_fst.set_F(PairState(q1, q2), fst1.ρ[q1] * fst2.ρ[q2])
    
    return comp_fst

### Subquestion e) - Lehmann's Algorithm

In [17]:
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()

        
    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta
    
    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        # TODO: implement
        R = self.W
        # print(len(self.W))
        # print(len(self.fsa.Q))
        # print(R[0])
        for j in range(len(self.fsa.Q)):
            tmp = R.copy()
            # print(tmp)
            for i in range(len(self.fsa.Q)):
                for k in range(len(self.fsa.Q)):
                    # print(f"i: {i}, j: {j}, k: {k}")
                    # print(tmp[j])
                    R[i][k] += tmp[i][j] * tmp[j][j].star() * tmp[j][k]
                    
        ps = dict()
        for i, q1 in enumerate(self.fsa.Q):
            for j, q2 in enumerate(self.fsa.Q):
                if i == j:
                    ps[(q1, q2)] = R[i][j] + self.R.one
                else:
                    ps[(q1, q2)] = R[i][j]
        return frozendict(ps)

    def _iterate(self, K):
        P = self.R.diag(self.N)
        for n in range(K):
            P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
            return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [25]:
for ex in examples:
    pathsum = Pathsum(ex['fst1']['fst'])
    print(pathsum.fixpoint())
    print(pathsum.lehmann())

frozendict.frozendict({(0, 0): 1.0, (0, 1): 0.25, (1, 0): 0.0, (1, 1): 1.146788990825688})
frozendict.frozendict({(0, 0): 1.0, (0, 1): 0.25, (1, 0): 0.0, (1, 1): 1.146788990825688})
frozendict.frozendict({(0, 0): Tropical(0.0), (0, 1): Tropical(35), (1, 0): Tropical(25), (1, 1): Tropical(0.0)})
frozendict.frozendict({(0, 0): Tropical(0), (0, 1): Tropical(35), (1, 0): Tropical(25), (1, 1): Tropical(0.0)})
frozendict.frozendict({(0, 0): True, (0, 1): True, (0, 2): True, (1, 0): True, (1, 1): True, (1, 2): True, (2, 0): True, (2, 1): True, (2, 2): True})
frozendict.frozendict({(0, 0): True, (0, 1): True, (0, 2): True, (1, 0): True, (1, 1): True, (1, 2): True, (2, 0): True, (2, 1): True, (2, 2): True})
frozendict.frozendict({(0, 0): 1.204687055617909, (0, 1): 0.049703748026656, (0, 2): 0.238528037012346, (1, 0): 0.423848011191222, (1, 1): 1.064607810824038, (1, 2): 0.083921906215862, (2, 0): 0.297575363071423, (2, 1): 0.220654517754702, (2, 2): 1.058919921888142})
frozendict.frozendict({(0

### Subquestion f) - Normalizer

We provide some test cases for you to inspect.

In [21]:
examples[0]['fst1']['Z'], examples[0]['fst2']['Z'], examples[0]['composition']['Z']

(0.00138425, 0.000997195294118, 7.07192371e-07)

In [22]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """
    
    pathsum = Pathsum(fst)
    # TODO: implement normalizer
    leh = pathsum.lehmann()
    norm = fst.R.zero
    for q1 in fst.Q:
        for q2 in fst.Q:
            norm += fst.λ[q1] * leh[(q1, q2)] * fst.ρ[q2]
    return norm

### Subquestion g) - Log-likelihood

In [23]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """

    # TODO: implement log-likelihood
    fst = edit_distance_fst(source_alphabet, target_alphabet)
    fst
    pathsum = Pathsum(fst)
    leh = pathsum.lehmann()
    logl = fst.R.zero
    for q1 in fst.Q:
        for q2 in fst.Q:
            for q3 in fst.Q:
                logl += fst.λ[q1] * leh[(q1, q2)] * fst.ρ[q3]
                
    print(normalizer(fst))
    return logl

In [24]:
log_likelihood({'a', 'b', 'c'}, {'d', 'e'}, "aa", "dd")

AttributeError: 'int' object has no attribute 'score'

### Subquestion h) - Decoding

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [ ]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    # TODO: implement
    raise NotImplementedError